# What is this noetbook?
This notebook contains working of GP with AL on all Dataset.

# **Correct AL model** ✅

## Step 1: Data Preparation ✅


### Installation


In [ ]:
!pip install useful_rdkit_utils tqdm scikit-learn 'modAL-python>=0.4.1'
!pip install gpytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6

### git clone repo


In [ ]:
!git clone https://github.com/meyresearch/ActiveLearning_BindingAffinity

Cloning into 'ActiveLearning_BindingAffinity'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 11), reused 6 (delta 3), pack-reused 0
Receiving objects: 100% (25/25), 255.89 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
### Import

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import gpytorch
import torch
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Function to convert SMILES to ECFP8 fingerprints


In [ ]:
# Function to convert SMILES to ECFP8 fingerprints
def smiles_to_ecfp8(smiles_df, smiles_column_name):
    fingerprints = []
    for smiles in tqdm(smiles_df[smiles_column_name], desc="Processing SMILES"):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=4, nBits=4096)
            fp_array = np.zeros((0,), dtype=np.int8)
            AllChem.DataStructs.ConvertToNumpyArray(fp, fp_array)
            fingerprints.append(fp_array)
    return np.array(fingerprints)

# File paths for datasets
file_paths = {
    'TYK2': '/content/ActiveLearning_BindingAffinity/TYK2_final.csv',
    'Mpro': '/content/ActiveLearning_BindingAffinity/Mpro_final.csv',
    'D2R': '/content/ActiveLearning_BindingAffinity/D2R_final.csv',
    'USP7': '/content/ActiveLearning_BindingAffinity/USP7_final.csv'
}

# Initialize dictionaries to store fingerprints and affinities
fingerprints = {}
affinities = {}
top_2p_values = {}
top_5p_values = {}

# Read datasets and process SMILES strings
for key, path in file_paths.items():
    df = pd.read_csv(path)
    fingerprints[key] = smiles_to_ecfp8(df, 'SMILES')
    affinities[key] = torch.tensor(df['affinity'].values, dtype=torch.float32)
    top_2p_values[key] = df['top_2p'].values ##aded this to get top_2%
    top_5p_values[key] = df['top_5p'].values
    print(f"{key} fingerprints shape:", fingerprints[key].shape)

Processing SMILES: 100%|██████████| 9997/9997 [00:08<00:00, 1180.97it/s]


TYK2 fingerprints shape: (9997, 4096)


Processing SMILES: 100%|██████████| 665/665 [00:00<00:00, 1873.23it/s]


Mpro fingerprints shape: (665, 4096)


Processing SMILES: 100%|██████████| 2502/2502 [00:02<00:00, 984.76it/s]


D2R fingerprints shape: (2502, 4096)


Processing SMILES: 100%|██████████| 4535/4535 [00:11<00:00, 405.78it/s]

USP7 fingerprints shape: (4535, 4096)


## Step 2: Active Learning Function ⛔


### Exploration strategy function ✅


In [ ]:
# Exploration strategy function
def explore_selection(train_x, batch_size):
    train_x_np = train_x.cpu().numpy()
    selected_indices = []
    remaining_indices = np.arange(len(train_x_np))
    selected_indices.append(np.random.choice(remaining_indices))
    remaining_indices = np.setdiff1d(remaining_indices, selected_indices)

    while len(selected_indices) < batch_size:
        # Euclidian distance
        distances = np.sum((train_x_np[remaining_indices] - train_x_np[selected_indices[-1]])**2, axis=1)
        next_index = remaining_indices[np.argmax(distances)] # Select farthest point
        selected_indices.append(next_index)
        remaining_indices = np.setdiff1d(remaining_indices, next_index) # Update remaining indices

    return np.array(selected_indices)

Either use above or below code.
Below is for UCB

### UCB ✅

In [ ]:
import numpy as np
import math

## UCB

def ucb_explore_selection(train_x, batch_size, c):
    train_x_np = train_x.cpu().numpy()  # Convert to NumPy array
    num_samples = len(train_x_np)
    selected_indices = []
    remaining_indices = np.arange(num_samples)  # All indices

    # Initialize counts and rewards
    counts = np.zeros(num_samples)
    rewards = np.zeros(num_samples)

    # Select initial random index
    initial_index = np.random.choice(remaining_indices)
    selected_indices.append(initial_index)
    remaining_indices = np.setdiff1d(remaining_indices, selected_indices)
    counts[initial_index] += 1

    while len(selected_indices) < batch_size:
        ucb_values = []

        for idx in remaining_indices:
            if counts[idx] == 0:
                ucb_value = float('inf')  # Ensure each action is tried at least once
            else:
                average_reward = rewards[idx] / counts[idx]
                ucb_value = average_reward + c * math.sqrt(math.log(len(selected_indices)) / counts[idx])
            ucb_values.append(ucb_value)

        next_index = remaining_indices[np.argmax(ucb_values)]
        selected_indices.append(next_index)
        remaining_indices = np.setdiff1d(remaining_indices, next_index)
        counts[next_index] += 1

        # Simulate reward (for demonstration purposes, using Euclidean distance as a proxy)
        last_selected_point = train_x_np[selected_indices[-2]]
        current_point = train_x_np[next_index]
        reward = -np.linalg.norm(current_point - last_selected_point)  # Negative distance as reward
        rewards[next_index] += reward

    return np.array(selected_indices)

### Function to perform active learning based on the protocol ⛔

In [ ]:
def active_learning(x, y, protocol, model_type, seed, dataset_name, protocol_name):
    np.random.seed(seed)
    torch.manual_seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x, y = x.to(device), y.to(device)

    selected_indices = []
    remaining_indices = np.array(list(range(len(x))))
    metrics_history = []
    uncertainties_history = []

    for iteration, (method, batch_size) in enumerate(protocol, 1):
        if method == "random":
            new_indices = np.random.choice(remaining_indices, batch_size, replace=False)
        elif method == "explore":
            new_indices = explore_selection(x[remaining_indices], batch_size)
        elif method == "exploit":
            model, likelihood, _, _ = train_and_evaluate(x[selected_indices], y[selected_indices], x[remaining_indices], y[remaining_indices])
            uncertainties = likelihood(model(x[remaining_indices])).variance.detach().cpu().numpy()
            new_indices = remaining_indices[np.argsort(uncertainties)[-batch_size:]]
            uncertainties_history.append(uncertainties)
        elif method == "ucb":
            # Select a batch of samples using the UCB strategy
            new_indices = ucb_explore_selection(x[remaining_indices], batch_size, c=1.0)  # Adjust `c` as needed

        selected_indices.extend(new_indices)
        remaining_indices = np.setdiff1d(remaining_indices, new_indices)

        visualize_sample_selection(x.cpu(), selected_indices, iteration, dataset_name, protocol_name, seed)

        train_x, test_x, train_y, test_y = train_test_split(x[selected_indices], y[selected_indices], test_size=0.2, random_state=42)
        final_model, final_likelihood, _, _ = train_and_evaluate(train_x, train_y, test_x, test_y)
        r2, spearman_corr, recall_2_percent, recall_5_percent, jaccard = calculate_metrics(final_model, final_likelihood, test_x, test_y)

        metrics_history.append({
            'r2': r2,
            'spearman': spearman_corr,
            'recall_2_percent': recall_2_percent,
            'recall_5_percent': recall_5_percent,
            'jaccard': jaccard
        })

    visualize_performance(metrics_history, dataset_name, protocol_name, seed)
    visualize_uncertainty(uncertainties_history, dataset_name, protocol_name, seed)

    return selected_indices, r2, spearman_corr, recall_2_percent, recall_5_percent, jaccard


## Step 3: Model Training and Evaluation


In [ ]:
class TanimotoKernel(gpytorch.kernels.Kernel):
    def __init__(self, **kwargs):
        super(TanimotoKernel, self).__init__(**kwargs)
        self.register_parameter(name="raw_lengthscale", parameter=torch.nn.Parameter(torch.zeros(1)))

    def forward(self, x1, x2, diag=False, **params):
        if diag:
            return self.covar_dist(x1, x2, diag=True, **params)
        else:
            x1_norm = x1.pow(2).sum(dim=-1, keepdim=True)
            x2_norm = x2.pow(2).sum(dim=-1, keepdim=True)
            x1_dot_x2 = x1 @ x2.transpose(-1, -2)
            denominator = x1_norm + x2_norm.transpose(-1, -2) - x1_dot_x2
            return x1_dot_x2 / denominator.clamp(min=1e-8)

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = TanimotoKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def train_and_evaluate(train_x, train_y, test_x, test_y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    model = GPRegressionModel(train_x, train_y, likelihood).to(device)

   # calculate_metrics

    model.train()
    likelihood.train()

    print("🌞🌞🌞")
    print(f" Satrted {dataset_name} with GP model using {protocol_name} protocol and seed {seed}.")
    print("🌊🌊🌊")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 200
    losses = []
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        losses.append(loss.item())
        if (i+1) % 100 == 0:
          print(f"Epoch {i+1} | Loss: {loss.item():.2f}")
        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        preds = likelihood(model(test_x))
        mean = preds.mean
        mse = torch.mean((mean - test_y) ** 2)
        print(f'MSE: {mse.item()}')

    return model, likelihood, losses, mean.cpu().numpy()

## Step 4: Metrics Calculation


In [ ]:
def calculate_metrics(model, likelihood, test_x, test_y):
    model.eval()
    likelihood.eval()
    with torch.no_grad():
        output_test = model(test_x.to(device))
        predictions = output_test.mean.cpu().numpy()

        r2 = r2_score(test_y.cpu().numpy(), predictions)
        spearman_corr, _ = spearmanr(test_y.cpu().numpy(), predictions)

        top_2_percent = int(0.02 * len(predictions))
        top_5_percent = int(0.05 * len(predictions))

        top_2_indices = np.argsort(predictions)[-top_2_percent:]
        top_5_indices = np.argsort(predictions)[-top_5_percent:]

        recall_2_percent = np.sum(test_y.cpu().numpy()[top_2_indices]) / np.sum(test_y.cpu().numpy())
        recall_5_percent = np.sum(test_y.cpu().numpy()[top_5_indices]) / np.sum(test_y.cpu().numpy())

        binary_predictions = predictions > np.median(predictions)
        binary_test_y = test_y.cpu().numpy() > np.median(test_y.cpu().numpy())
        jaccard = jaccard_score(binary_test_y, binary_predictions)

        print(f"R2: {r2}")
        print(f"Spearman Correlation: {spearman_corr}")
        print(f"Recall 2%: {recall_2_percent}")
        print(f"Recall 5%: {recall_5_percent}")
        print(f"Jaccard Score: {jaccard}")

        return r2, spearman_corr, recall_2_percent, recall_5_percent,jaccard

## Visualisation

### Step 1: Sample Selection Visualization


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def visualize_sample_selection(x, selected_indices, iteration, dataset_name, protocol_name, seed):
    pca = PCA(n_components=2)
    x_2d = pca.fit_transform(x)

    plt.figure(figsize=(10, 6))
    plt.scatter(x_2d[:, 0], x_2d[:, 1], c='lightgray', label='Unselected Samples')
    plt.scatter(x_2d[selected_indices, 0], x_2d[selected_indices, 1], c='red', label='Selected Samples')
    plt.title(f'Sample Selection at Iteration {iteration}\nDataset: {dataset_name}, Protocol: {protocol_name}, Seed: {seed}')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.legend()
    plt.savefig(f'sample_selection_{dataset_name}_{protocol_name}_{seed}_iter{iteration}.png')
    plt.show()

### Step 2: Performance Metrics Visualization


In [ ]:
def visualize_performance(metrics_history, dataset_name, protocol_name, seed):
    iterations = list(range(1, len(metrics_history) + 1))
    r2_scores = [metrics['r2'] for metrics in metrics_history]
    spearman_corrs = [metrics['spearman'] for metrics in metrics_history]

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.bar(iterations, r2_scores)
    plt.title(f'R2 Score over Iterations\nDataset: {dataset_name}, Protocol: {protocol_name}, Seed: {seed}')
    plt.xlabel('Iteration')
    plt.ylabel('R2 Score')

    plt.subplot(1, 2, 2)
    plt.bar(iterations, spearman_corrs)
    plt.title(f'Spearman Correlation over Iterations\nDataset: {dataset_name}, Protocol: {protocol_name}, Seed: {seed}')
    plt.xlabel('Iteration')
    plt.ylabel('Spearman Correlation')

    plt.tight_layout()
    plt.savefig(f'performance_metrics_{dataset_name}_{protocol_name}_{seed}.png')
    plt.show()

### Step 3: Uncertainty Visualization


In [ ]:
def visualize_uncertainty(uncertainties_history, dataset_name, protocol_name, seed):
    iterations = list(range(1, len(uncertainties_history) + 1))
    avg_uncertainties = [np.mean(uncertainties) for uncertainties in uncertainties_history]

    plt.figure(figsize=(10, 6))
    plt.plot(iterations, avg_uncertainties, marker='o')
    plt.title(f'Average Uncertainty over Iterations\nDataset: {dataset_name}, Protocol: {protocol_name}, Seed: {seed}')
    plt.xlabel('Iteration')
    plt.ylabel('Average Uncertainty')
    plt.savefig(f'uncertainty_{dataset_name}_{protocol_name}_{seed}.png')
    plt.show()

## Step 5: Main Execution


### Define selection protocols


In [ ]:
selection_protocols = {
    "random-exploit": [("random", 60)] + [("exploit", 30)]*10,
    "random-explore-exploit": [("random", 60)] + [("explore", 30)]*2 + [("exploit", 30)]*8,
    "random-ucb": [("random", 60)] + [("ucb", 30)]*10,
}

#can be any random number for testing keep one.
random_seeds = [750]

# Initialize DataFrame to store results
results_df = pd.DataFrame(columns=["Dataset", "Model", "Protocol", "Seed", "R2", "Spearman", "Recall_2%", "Recall_5%", "Jaccard"])

### main execution

In [ ]:
from tqdm import tqdm
import torch
import pandas as pd

results_df = pd.DataFrame()

for dataset_name in tqdm(file_paths.keys(), desc="Datasets"):
    x = torch.tensor(fingerprints[dataset_name], dtype=torch.float32)
    y = affinities[dataset_name]
    for protocol_name, protocol in tqdm(selection_protocols.items(), desc="Protocols", leave=False):
        for seed in tqdm(random_seeds, desc="Seeds", leave=False):
            selected_indices, r2, spearman_corr, recall_2_percent, recall_5_percent, jaccard = active_learning(x, y, protocol, "GP", seed, dataset_name, protocol_name)
            new_row = pd.DataFrame({
                "Dataset": [dataset_name],
                "Model": ["GP"],
                "Protocol": [protocol_name],
                "Seed": [seed],
                "R2": [r2],
                "Spearman": [spearman_corr],
                "Recall_2%": [recall_2_percent],
                "Recall_5%": [recall_5_percent],
                "Jaccard": [jaccard]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)
            print("🌞🌞🌞")
            print(f" Finished {dataset_name} with GP model using {protocol_name} protocol and seed {seed}.")
            print("🌊🌊🌊")


# Save results to CSV
results_df.to_csv("active_learning_results.csv", index=False)



Protocols:   0%|          | 0/3 [00:00<?, ?it/s]

Seeds:   0%|          | 0/1 [00:00<?, ?it/s]

🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.78
Epoch 200 | Loss: 1.76
MSE: 1.0814915895462036
R2: 0.054022731534526636
Spearman Correlation: 0.27272727272727276
Recall 2%: 1.0000001192092896
Recall 5%: 1.0000001192092896
Jaccard Score: 0.5
🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.72
Epoch 200 | Loss: 1.71
MSE: 1.5109660625457764
🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.77
Epoch 200 | Loss: 1.75
MSE: 1.553364634513855
R2: 0.1350410254398603
Spearman Correlation: 0.29205366357069146
Recall 2%: 1.0000001192092896
Recall 5%: 1.0000001192092896
Jaccard Score: 0.38461538461538464
🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.74
Epoch 200 | Loss: 1.73
MSE: 1.4455077648162842
🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Los

/usr/local/lib/python3.10/dist-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.70
Epoch 200 | Loss: 1.69
MSE: 1.5168694257736206
R2: 0.4636418917296121
Spearman Correlation: 0.7408042691061558
Recall 2%: 0.025392750278115273
Recall 5%: 0.05237443372607231
Jaccard Score: 0.5882352941176471
🌞🌞🌞
 Satrted TYK2 with GP model using random-exploit protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.68
Epoch 200 | Loss: 1.68
MSE: 0.8680717945098877


/usr/local/lib/python3.10/dist-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


...

🌞🌞🌞
 Finished USP7 with GP model using random-explore-exploit protocol and seed 750.
🌊🌊🌊




Seeds:   0%|          | 0/1 [00:00<?, ?it/s]

🌞🌞🌞
 Satrted USP7 with GP model using random-ucb protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 2.01
Epoch 200 | Loss: 1.39
MSE: 1.4692144393920898
R2: 0.1477973995440115
Spearman Correlation: 0.37391224581226906
Recall 2%: 0.9999999403953552
Recall 5%: 0.9999999403953552
Jaccard Score: 0.5
🌞🌞🌞
 Satrted USP7 with GP model using random-ucb protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.74
Epoch 200 | Loss: 1.39
MSE: 0.5954294204711914
R2: 0.5945027946663495
Spearman Correlation: 0.7632546901155375
Recall 2%: 1.0000001192092896
Recall 5%: 1.0000001192092896
Jaccard Score: 0.6363636363636364
🌞🌞🌞
 Satrted USP7 with GP model using random-ucb protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 1.10
Epoch 200 | Loss: 0.44
MSE: 0.4717293679714203
R2: 0.665460531353455
Spearman Correlation: 0.8137983655382647
Recall 2%: 0.9999999403953552
Recall 5%: 0.05060647800564766
Jaccard Score: 0.8461538461538461
🌞🌞🌞
 Satrted USP7 with GP model using random-ucb protocol and seed 750.
🌊🌊🌊
Epoch 100 | Loss: 0.05
Epo



Seeds: 100%|██████████| 1/1 [00:49<00:00, 49.38s/it]

                                                    
Datasets: 100%|██████████| 4/4 [14:51<00:00, 222.83s/it]

🌞🌞🌞
 Finished USP7 with GP model using random-ucb protocol and seed 750.
🌊🌊🌊


In [ ]:
#df_with_fingerprint
file_path = "/content/active_learning_results.csv"
df = pd.read_csv(file_path)


In [ ]:
df

,Dataset,Model,Protocol,Seed,R2,Spearman,Recall_2%,Recall_5%,Jaccard
0,TYK2,GP,random-exploit,750,0.634011,0.764744,0.016007,0.055330,0.674419
1,TYK2,GP,random-explore-exploit,750,0.497197,0.732008,0.017220,0.050367,0.600000
2,TYK2,GP,random-ucb,750,0.583870,0.831832,0.018704,0.052591,0.756098
3,Mpro,GP,random-exploit,750,0.165938,0.585960,0.012866,0.043797,0.565217
4,Mpro,GP,random-explore-exploit,750,0.388470,0.666784,0.017319,0.049349,0.636364
5,Mpro,GP,random-ucb,750,0.890810,0.915977,0.016771,0.050312,0.800000
6,D2R,GP,random-exploit,750,0.274773,0.508514,0.016963,0.053527,0.469388
7,D2R,GP,random-explore-exploit,750,0.300723,0.436344,0.015874,0.051085,0.428571
8,D2R,GP,random-ucb,750,0.962845,0.947838,0.021245,0.063734,0.894737
9,USP7,GP,random-exploit,750,0.781901,0.875584,0.016653,0.049454,0.894737
